In [1]:
import re
import json
import pycld2

from time import time, sleep
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from random import randint
from transformers import *
from deep_translator import GoogleTranslator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
sentiment_model = SentimentIntensityAnalyzer()

: 

In [ ]:
paraphrase_model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
paraphrase_tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

In [4]:
translation_target = 'en'
translation_original = 'auto'
sumarizer_stopwords = set(stopwords.words("english"))

In [6]:
def text_spliter(text:str = ''):
    normalize = re.sub(
        r'\[(\d+)\]|[^\w\s\.\:\,\(\)]\/\\\$\%\@\#\!\?\.\&\^\*\-\+\=\_', '', text
    )
    splited = re.split(r'\.\s', normalize)
    results = []
    
    for item in splited:
        item = re.sub(r'\.$', '', item.strip().capitalize()) + '.'
        results.append({
            'text': item,
            'count': len(item.split()),
            'length': len(item),
            'size': len(item.encode('utf-8'))
        })
    
    return results

def translateToTarget(text: str = ''):
    start = time()
    _, textBytesFound, details = pycld2.detect(text)
    translated = GoogleTranslator(source=translation_original, target=translation_target).translate(text.lower())

    return {
        'size': textBytesFound,
        'detected': details[0],
        'translated': translated,
        'elapsed': time() - start,
    }

def translateToOriginal(translated: str = '', translation_original: str = 'en'):
    start = time()
    original = GoogleTranslator(source=translation_target, target=translation_original).translate(translated.lower())

    return {
        'translated': original,
        'language': translation_original,
        'elapsed': time() - start,
    }

def sentiment(content: str = '', original_language: str = 'en'):
    start = time()
    score = sentiment_model.polarity_scores(content)
    
    return {
        'sentences': translateToOriginal(content, original_language),
        'polarity': score,
        'elapsed': time() - start,
    }

def summarization(content: str = '', original_language: str = 'en'):
    words = word_tokenize(content)
    sentences = sent_tokenize(content)

    freqTable = dict()
    sentenceValue = dict()

    for word in words:
        word = word.lower()
        if word in sumarizer_stopwords:
            continue
        if word in freqTable:
            freqTable[word] = 1
        else:
            freqTable[word] = 1 

    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else: sentenceValue[sentence] = freq 

        sumValues = 0

        for sentence in sentenceValue:
            sumValues += sentenceValue[sentence] 

    average = int(sumValues / len(sentenceValue)) 

    summary = ''
    
    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (0.9 * average)):
            summary += sentence
        
    
    return translateToOriginal(summary, original_language)

def paraphrasedLine(sentence, num_return_sequences=5, num_beams=5):
    inputs = paraphrase_tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
    outputs = paraphrase_model.generate(
        **inputs,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
    )

    return paraphrase_tokenizer.batch_decode(outputs, skip_special_tokens=True)

def paraphrased(content: str = '', original_language: str = 'en'):
    start = time()
    paraphrases = paraphrasedLine(content, num_beams=5, num_return_sequences=5)
#     result = paraphrases[randint(0, 2)]
#     results = {
#         'translated': translateToOriginal(result, original_language),
#         'elapsed': time() - start
#     }
        
    print('elapsed', time() - start)
    return paraphrases

In [7]:
source_content = 'Izaac Hindom adalah seorang birokrat dan politikus Indonesia yang menjabat sebagai Gubernur Irian Jaya (sekarang Papua) dari tahun 1982 sampai 1988. Lahir di Fakfak, Hindom memulai pendidikannya di sekolah Belanda untuk orang asli Papua dan menempuh pendidikan kejuruan dalam bidang kepamongprajaan. Hindom bekerja dalam berbagai jabatan pegawai negeri di tingkat distrik pada masa Nugini Belanda. Ia juga mencalonkan diri sebagai anggota Dewan Nugini tetapi tidak terpilih. Setelah Nugini Belanda menjadi bagian dari Indonesia, Hindom diangkat menjadi anggota Badan Pemerintahan Harian pada tahun 1965 dan terpilih menjadi anggota Dewan Perwakilan Rakyat selama dua periode pada tahun 1971 dan 1977. Dia mengakhiri masa jabatan keduanya sebagai anggota Dewan setelah diangkat sebagai wakil gubernur pada tahun 1980 dan menjadi gubernur dua tahun kemudian setelah pendahulunya meninggal.'

In [8]:
splited = text_spliter(source_content)

print(json.dumps(splited, indent = 2))

[
  {
    "text": "Izaac hindom adalah seorang birokrat dan politikus indonesia yang menjabat sebagai gubernur irian jaya (sekarang papua) dari tahun 1982 sampai 1988.",
    "count": 21,
    "length": 148,
    "size": 148
  },
  {
    "text": "Lahir di fakfak, hindom memulai pendidikannya di sekolah belanda untuk orang asli papua dan menempuh pendidikan kejuruan dalam bidang kepamongprajaan.",
    "count": 20,
    "length": 150,
    "size": 150
  },
  {
    "text": "Hindom bekerja dalam berbagai jabatan pegawai negeri di tingkat distrik pada masa nugini belanda.",
    "count": 14,
    "length": 97,
    "size": 97
  },
  {
    "text": "Ia juga mencalonkan diri sebagai anggota dewan nugini tetapi tidak terpilih.",
    "count": 11,
    "length": 76,
    "size": 76
  },
  {
    "text": "Setelah nugini belanda menjadi bagian dari indonesia, hindom diangkat menjadi anggota badan pemerintahan harian pada tahun 1965 dan terpilih menjadi anggota dewan perwakilan rakyat selama dua periode pada t

In [9]:
english_content = translateToTarget(source_content)

english_content

{'size': 849,
 'detected': ('INDONESIAN', 'id', 99, 1101.0),
 'translated': "Izaac Hindum is an Indonesian bureaucrat and politician who served as governor of Irian Jaya (now Papua) from 1982 to 1988. Born in Fakfak, Hindum began his education at a Dutch school for Papuans and pursued a vocational education in civil service. Hindum worked in various civil service positions at the district level during the Dutch New Guinea era. He also ran for the New Guinea council but was not elected. After Dutch New Guinea became part of Indonesia, Hindom was appointed a member of the daily governing body in 1965 and was elected to the People's Representative Council for two terms in 1971 and 1977. He ended his second term as a member of the council after being appointed deputy governor in 1980. and became governor two years later after his predecessor died.",
 'elapsed': 1.653182029724121}

In [10]:
translated_content = english_content['translated']
original_language = english_content['detected'][1]

In [11]:
souce_content = translateToOriginal(translated_content, original_language)

souce_content

{'translated': 'izaac hindum adalah seorang birokrat dan politisi indonesia yang menjabat sebagai gubernur irian jaya (sekarang papua) dari tahun 1982 hingga 1988. lahir di fakfak, hindum memulai pendidikannya di sekolah belanda untuk orang papua dan menempuh pendidikan kejuruan di bidang pamong praja. hindum bekerja di berbagai posisi pegawai negeri di tingkat kabupaten selama era nugini belanda. dia juga mencalonkan diri sebagai dewan guinea baru tetapi tidak terpilih. setelah nugini belanda menjadi bagian dari indonesia, hindom diangkat menjadi anggota harian pada tahun 1965 dan terpilih menjadi dewan perwakilan rakyat untuk dua periode pada tahun 1971 dan 1977. ia mengakhiri masa jabatan keduanya sebagai anggota dewan setelah diangkat menjadi wakil gubernur pada tahun 1980. dan menjadi gubernur dua tahun kemudian setelah pendahulunya meninggal.',
 'language': 'id',
 'elapsed': 0.5546519756317139}

In [12]:
for line in translated_content.split('. '):
    print(json.dumps(sentiment(line, original_language), indent = 2))

{
  "sentences": {
    "translated": "izaac hindum adalah seorang birokrat dan politisi Indonesia yang menjabat sebagai gubernur irian jaya (sekarang papua) dari tahun 1982 hingga 1988",
    "language": "id",
    "elapsed": 0.8052318096160889
  },
  "polarity": {
    "neg": 0.0,
    "neu": 1.0,
    "pos": 0.0,
    "compound": 0.0
  },
  "elapsed": 0.8067431449890137
}
{
  "sentences": {
    "translated": "lahir di fakfak, hindum memulai pendidikannya di sekolah belanda untuk orang papua dan menempuh pendidikan kejuruan di pamong praja",
    "language": "id",
    "elapsed": 0.5193290710449219
  },
  "polarity": {
    "neg": 0.0,
    "neu": 1.0,
    "pos": 0.0,
    "compound": 0.0
  },
  "elapsed": 0.5195009708404541
}
{
  "sentences": {
    "translated": "hindum bekerja di berbagai posisi pegawai negeri di tingkat kabupaten selama era nugini belanda",
    "language": "id",
    "elapsed": 0.5915110111236572
  },
  "polarity": {
    "neg": 0.0,
    "neu": 1.0,
    "pos": 0.0,
    "compoun

In [13]:
summarized_text = summarization("Izaac Hindum is an Indonesian bureaucrat and politician who served as governor of Irian Jaya (now Papua) from 1982 to 1988. Born in Fakfak, Hindum began his education at a Dutch school for Papuans and pursued a vocational education in civil service. Hindum worked in various civil service positions at the district level during the Dutch New Guinea era. He also ran for the New Guinea council but was not elected. After Dutch New Guinea became part of Indonesia, Hindom was appointed a member of the daily governing body in 1965 and was elected to the People's Representative Council for two terms in 1971 and 1977. He ended his second term as a member of the council after being appointed deputy governor in 1980. and became governor two years later after his predecessor died.", 'id')

print(json.dumps(summarized_text, indent = 2))

{
  "translated": "izaac hindum adalah seorang birokrat dan politikus indonesia yang menjabat sebagai gubernur irian jaya (sekarang papua) dari tahun 1982 hingga 1988.lahir di fakfak, hindum memulai pendidikannya di sekolah belanda untuk orang papua dan menempuh pendidikan kejuruan di bidang pamong praja.setelah belanda baru guinea menjadi bagian dari indonesia, hindom diangkat menjadi anggota badan pengurus harian pada tahun 1965 dan terpilih menjadi dewan perwakilan rakyat untuk dua periode pada tahun 1971 dan 1977. ia mengakhiri masa jabatan keduanya sebagai anggota dewan setelah diangkat menjadi wakil gubernur pada tahun 1980. dan menjadi gubernur dua tahun kemudian setelah pendahulunya meninggal.",
  "language": "id",
  "elapsed": 1.238706111907959
}


In [15]:
paraphrased_text = paraphrased("Izaac Hindum is an Indonesian bureaucrat and politician who served as governor of Irian Jaya (now Papua) from 1982 to 1988 Born in Fakfak, Hindum began his education at a Dutch school for Papuans and pursued a vocational education in civil service", 'id')

print(json.dumps(paraphrased_text, indent = 2))

elapsed 4.768844127655029
[
  "Izaac Hindum was the governor of Irian Jaya from 1982 to 1988 and was a civil servant.",
  "Izaac Hindum is an Indonesian bureaucrat and politician who served as governor of Irianjaya from 1982 to 1988",
  "Izaac Hindum was the governor of Irian Jaya from 1982 to 1988 and was an Indonesian bureaucrat and politician.",
  "Izaac Hindum was the governor of Irian Jaya from 1982 to 1988 and was an Indonesian bureaucrat.",
  "Izaac Hindum is an Indonesian bureaucrat and politician who served as governor of Irianjaya from 1982 to 1988."
]
